In [ ]:
# CNN을 이용하여 개, 고양이 이미지 분류 (이항분류)

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import os
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as util
from keras.losses import BinaryCrossentropy

data_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
pathto_zip = util.get_file('cat_and_dogs.zip', origin=data_url, extract=True)
PATH = os.path.join(os.path.dirname(pathto_zip), 'cats_and_dogs_filtered')
# print(PATH)

batch_size = 128
epochs=15
IMG_HEIGHT =150
IMG_WIDTH =150

# 데이터 준비
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# print(train_cats_dir)
# /Users/heojunho/.keras/datasets/cats_and_dogs_filtered/train/cats

num_cats_dir = len(os.listdir(path=train_cats_dir))	# 1000
num_dogs_dir = len(os.listdir(path=train_dogs_dir))	# 1000
# print(num_dogs_dir)
num_cats_val = len(os.listdir(path=validation_cats_dir))	# 500
num_dogs_val = len(os.listdir(path=validation_dogs_dir))	# 500
# print(num_dogs_val)

total_train = num_cats_dir + num_dogs_dir		# 2000
total_val = num_dogs_val + num_cats_val			# 1000
# print(total_train)
# print(total_val)



# 데이터를 실수 타입의 텐서로 전처리
# 두 개의 이미지에 대한 라벨링

train_image_gen = ImageDataGenerator(rescale=1./255)
validation_image_gen = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_gen.flow_from_directory(
	directory=train_dir,
	target_size=(IMG_HEIGHT, IMG_WIDTH),
	batch_size=batch_size,
	shuffle=True,
	class_mode='binary')
# (128, 150, 150, 3)

validation_data_gen = validation_image_gen.flow_from_directory(
	directory=validation_dir,
	target_size=(IMG_HEIGHT, IMG_WIDTH),
	batch_size=batch_size,
	shuffle=True,
	class_mode='binary')
# (128, 150, 150, 3)
for a,b in train_data_gen:
	print(a.shape, b.shape)
	print(b[0], b[1])
	break
for a,b in validation_data_gen:
	break

# 데이터 확인
sample_training_images,_ = next(train_data_gen)

def plotImage(img_arr):
	fig, axes = plt.subplots(1,5, figsize=(20,20))
	axes = axes.flatten()
	for img,ax in zip(img_arr, axes):
		ax.axis('off')
		ax.imshow(img)
		plt.tight_layout()
	plt.show()
plotImage(sample_training_images[:5])



# model
model = Sequential([
	Conv2D(32, (3, 3), input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), padding='same', activation='relu'),
	MaxPooling2D(pool_size=(2, 2)),
	Dropout(rate=0.2),
	Flatten(),
	Dense(units=128, activation='relu'),
	Dense(units=64, activation='relu'),
	Dense(units=1, activation='sigmoid'),  # 이진 분류에서는 시그모이드 활성화 함수 사용
])

model.compile(optimizer='adam', loss=BinaryCrossentropy(from_logits=False), metrics=['accuracy'])
model.summary()

# 학습
history = model.fit(
	train_data_gen,
	steps_per_epoch=total_train // batch_size,
	epochs=epochs,
	validation_data=validation_data_gen,
	validation_steps=total_val // batch_size,
)

model.save('cat_dog.keras')

# 학습 결과 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='accuracy')
plt.plot(epochs_range, val_acc, label='val_accuracy')
plt.legend(loc='best')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='loss')
plt.plot(epochs_range, val_loss, label='val_loss')
plt.legend(loc='best')
plt.show()


In [ ]:
#새로운 이미지를 분류
import numpy as np
from google.colab import files
import tensorflow as tf

mymodel=tf.keras.models.load_model('cat_dog.keras')
uploaded=files.upload()
print(uploaded.keys())

for fn in uploaed.keys():
    path='/content/'+ fn
    img = tf.keras.utils.load_img(path, target_size=(150,150))
    x=tf.keras.utils
